<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 10. Бустинг
## <center>Часть 3. Xgboost, стандартный Python-интерфейс

## Загрузка бибилиотек

In [15]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Загрузка и подготовка данных

Посмотрим на примере данных по оттоку клиентов из телеком-компании.

> **Важно**: XGBoost поддерживает только числовые признаки. Поэтому текстовые признаки требуют предобработки

Для обучения с Xgboost данные должны быть представлены в виде объекта `DMatrix`, предназначенного для представления разреженных данных. Его можно инициализировать через:
- текстовый формат libsvm,
-  Numpy 2D array (чаще всего)
- бинарный буфер-файл XGBoost

**Загрузим данные и осуществим минимальную предобработку.**

In [ ]:
# df = pd.read_csv("../../data/telecom_churn.csv")
df = pd.read_csv('https://github.com/Yorko/mlcourse.ai/raw/main/data/telecom_churn.csv')

In [ ]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


**Штаты просто занумеруем (хотя можно и лучше поступить), а признаки International plan (наличие международного роуминга), Voice mail plan (наличие голосовой почтыы) и целевой Churn сделаем бинарными.**

In [ ]:
state_enc = LabelEncoder()
df["State"] = state_enc.fit_transform(df["State"])
df["International plan"] = (df["International plan"] == "Yes").astype("int")
df["Voice mail plan"] = (df["Voice mail plan"] == "Yes").astype("int")
df["Churn"] = (df["Churn"]).astype("int")

**Разделим данные на обучающую и тестовую выборки в отношении 7:3. Инициализируем соотв. объекты DMatrix dtrain и dtest.**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Churn", axis=1),
    df["Churn"],
    test_size=0.3,
    stratify=df["Churn"],
    random_state=17,
)
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

**Посмотрим на статистику полученных объектов:**

In [ ]:
print(
    "Train dataset contains {0} rows and {1} columns".format(
        dtrain.num_row(), dtrain.num_col()
    )
)
print(
    "Test dataset contains {0} rows and {1} columns".format(
        dtest.num_row(), dtest.num_col()
    )
)

Train dataset contains 2333 rows and 19 columns
Test dataset contains 1000 rows and 19 columns


In [ ]:
print("Train mean target: ")
print(np.mean(dtrain.get_label()))

print("\nTest mean target: ")
print(np.mean(dtest.get_label()))

Train mean target: 
0.14487784

Test mean target: 
0.145


### Инициализация параметров

- бинарная классификация (`'objective':'binary:logistic'`)
- ограничим глубину деревьев (`'max_depth':3`)
- не хотим лишнего вывода (`'silent':1`)
- проведем 50 итераций бустинга
- шаг градиентного спуска довольно большой (`'eta':1`) - алгоритм будет обучаться быстро и "агрессивно" (лучше результаты будут, если уменьшить eta и увеличить число итераций)


In [ ]:
params = {"objective": "binary:logistic", "max_depth": 3, "silent": 1, "eta": 1}

num_rounds = 50

### Обучение классификатора
Тут мы просто передаем слоавть параметров, данные и число итераций.

In [ ]:
xgb_model = xgb.train(params, dtrain, num_rounds)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:19:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


**С помощью `watchlist` отслеживать качество алгоритма на тестовой выборке для каждой итерации.**

In [ ]:
watchlist = [(dtest, "test"), (dtrain, "train")]  # native interface only
xgb_model = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-logloss:0.32064	train-logloss:0.26994
[1]	test-logloss:0.26260	train-logloss:0.19868
[2]	test-logloss:0.26039	train-logloss:0.18454
[3]	test-logloss:0.26034	train-logloss:0.17782
[4]	test-logloss:0.25921	train-logloss:0.16673
[5]	test-logloss:0.23795	train-logloss:0.13979
[6]	test-logloss:0.22813	train-logloss:0.12606
[7]	test-logloss:0.22793	train-logloss:0.11946
[8]	test-logloss:0.23468	train-logloss:0.11390
[9]	test-logloss:0.22084	train-logloss:0.10775
[10]	test-logloss:0.22610	train-logloss:0.10243
[11]	test-logloss:0.23630	train-logloss:0.09749
[12]	test-logloss:0.23572	train-logloss:0.09590
[13]	test-logloss:0.23599	train-logloss:0.09359
[14]	test-logloss:0.23814	train-logloss:0.08856
[15]	test-logloss:0.24329	train-logloss:0.08494
[16]	test-logloss:0.24425	train-logloss:0.08285
[17]	test-logloss:0.24605	train-logloss:0.07929
[18]	test-logloss:0.24692	train-logloss:0.07582
[19]	test-logloss:0.25293	train-logloss:0.07244
[20]	test-logloss:0.26131	train-logloss:0.06767
[2

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:19:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[28]	test-logloss:0.26129	train-logloss:0.04711
[29]	test-logloss:0.26082	train-logloss:0.04598
[30]	test-logloss:0.26663	train-logloss:0.04470
[31]	test-logloss:0.27035	train-logloss:0.04297
[32]	test-logloss:0.27200	train-logloss:0.04234
[33]	test-logloss:0.27015	train-logloss:0.04160
[34]	test-logloss:0.27279	train-logloss:0.03967
[35]	test-logloss:0.27058	train-logloss:0.03766
[36]	test-logloss:0.27459	train-logloss:0.03649
[37]	test-logloss:0.27627	train-logloss:0.03476
[38]	test-logloss:0.27717	train-logloss:0.03404
[39]	test-logloss:0.27838	train-logloss:0.03251
[40]	test-logloss:0.28208	train-logloss:0.03123
[41]	test-logloss:0.28021	train-logloss:0.03009
[42]	test-logloss:0.28126	train-logloss:0.02891
[43]	test-logloss:0.28085	train-logloss:0.02759
[44]	test-logloss:0.28116	train-logloss:0.02660
[45]	test-logloss:0.28408	train-logloss:0.02571
[46]	test-logloss:0.27857	train-logloss:0.02437
[47]	test-logloss:0.28334	train-logloss:0.02289
[48]	test-logloss:0.28538	train-logloss:

### Прогнозы для тестовой выборки

In [ ]:
preds_prob = xgb_model.predict(dtest)

**Посчитаем долю правильных ответов алгоритма на тестовой выборке.**

In [ ]:
predicted_labels = preds_prob > 0.5
print(
    "Accuracy and F1 on the test set are: {} and {}".format(
        round(accuracy_score(y_test, predicted_labels), 3),
        round(f1_score(y_test, predicted_labels), 3),
    )
)

Accuracy and F1 on the test set are: 0.935 and 0.751
